In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd

from NLP_Natural_Disasters.data import get_data, clean_data

2022-04-07 16:48:43.542180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-07 16:48:43.542372: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
pd.set_option('display.max_colwidth', None)

Load the dataframe train.csv cleaned :

In [3]:
train = clean_data(get_data())

In [4]:
y_train = train['target']
X_train = train['text']
X_train_with_id = train.drop(columns='target')

In [5]:
X_train.head()

0                                               deed reason earthquake may allah forgive u
1                                                    forest fire near la ronge sask canada
2    resident asked shelter place notified officer evacuation shelter place order expected
3                                      people receive wildfire evacuation order california
4                                   got sent photo ruby alaska smoke wildfire pours school
Name: text, dtype: object

In [6]:
X_train[1]

'forest fire near la ronge sask canada'

Later fine-tune : Spell check is computationally expensive : impouve data cleaning for the train : make better prediction ?

In [8]:
#install autocorrect
!pip install autocorrect
from autocorrect import Speller 

#create function to spell check strings
def spell_check(x):
    spell = Speller(lang='en')
    return " ".join([spell(i) for i in x.split()])

#showcase spellcheck 
mispelled = 'Pleaze spelcheck this sentince'
spell_check(mispelled)

'Please spelcheck this sentence'

In [9]:
# X_train = spell_check(X_train)

Apply changes to test.csv

In [7]:
X_test = pd.read_csv('/home/dianavo/code/AxelCatelan/NLP_Natural_Disasters/raw_data/test.csv')
X_test = clean_data(X_test)

In [8]:
X_test.tail()

,id,text
3258,10861,earthquake safety los angeles ûò safety fastener xrwn
3259,10865,storm ri worse last hurricane cityampothers hardest hit yard look like bombed around k still without power
3260,10868,green line derailment chicago
3261,10874,meg issue hazardous weather outlook hwo
3262,10875,cityofcalgary activated municipal emergency plan yycstorm


In [12]:
#  X_test = spell_check(X_test)

Should be removed before predict on it ?

In [13]:
# 'uû÷ªòóåáåáåáï¡ì©ê'
# delete if a character or 2 letters is repeated 3times : jesssssssee , tltltltl

In [27]:
# tokenize the vocabulary 
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)
print(f'There are {vocab_size} different words in your corpus')
X_token = tk.texts_to_sequences(X_train)

# # Padding inputs : feed in the same tensor length
# X_pad = pad_sequences(X_token, dtype='float32', padding='post')
    
# X_pad, y_train, vocab_size

There are 15807 different words in your corpus


In [15]:
pip install gensim python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gensim.downloader as api

In [28]:
# load a word2vec embedding
word2vec_transfer = api.load("glove-wiki-gigaword-50")

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=35)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=35)

In [29]:
X_train_pad_2.shape
# nb token,pad_sequence size : nb words , 50 : embedding size

(7613, 35, 50)

In [17]:
# OR ??
# from gensim.models import Word2Vec

# word2vec = Word2Vec(sentences=X_train)
# wv = word2vec.wv

In [30]:
vocab_size

15807

In [31]:
%%time
# max length of char in a tweet
maxlen=35
embedding_size = 50

def build_model_rnn():
    model = Sequential()
    
    model.add(layers.Embedding(
        input_dim= vocab_size+1,
        input_length=maxlen,
        output_dim=embedding_size,
        mask_zero=True
    ))
    model.add(layers.LSTM(10, return_sequences=True))
    model.add(layers.LSTM(10))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dropout(0.15))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])
    return model

model_rnn = build_model_rnn()
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 35, 50)            790400    
                                                                 
 lstm_4 (LSTM)               (None, 35, 10)            2440      
                                                                 
 lstm_5 (LSTM)               (None, 10)                840       
                                                                 
 dense_4 (Dense)             (None, 30)                330       
                                                                 
 dropout_2 (Dropout)         (None, 30)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 794,041
Trainable params: 794,041
Non-tr

In [25]:
from tensorflow.keras.callbacks import EarlyStopping

In [32]:
es = EarlyStopping(patience=2, restore_best_weights=True)

model_cnn = build_model_rnn()

history = model_cnn.fit(X_train_pad_2, y_train, 
          validation_split=0.3,
          epochs=10, 
          batch_size=16,
          callbacks=[es], verbose=1)
print("Test Loss", history[0])
print("Test Accuracy", history[1])

Epoch 1/10


ValueError: in user code:

    File "/home/dianavo/.pyenv/versions/3.8.12/envs/nlp_project/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/dianavo/.pyenv/versions/3.8.12/envs/nlp_project/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/dianavo/.pyenv/versions/3.8.12/envs/nlp_project/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/dianavo/.pyenv/versions/3.8.12/envs/nlp_project/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/dianavo/.pyenv/versions/3.8.12/envs/nlp_project/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(fi

Prediction👌😍

In [ ]:
# 4) prediction
y_new = model.predict(X_new)